In [1]:
using Pkg
Pkg.activate("Dev")
using Skylight
using BenchmarkTools

  Activating project at `~/repos/Skylight.jl/run/Dev`


In [100]:
spacetime = SchwarzschildSpacetimeSphericalCoordinates(M=1.0)
camera = ImagePlane(distance = 200.0,
                         observer_inclination_in_degrees = 5,
                         horizontal_side = 20.0,
                         vertical_side = 20.0,
                         horizontal_number_of_pixels = 50,
                         vertical_number_of_pixels = 50)
model = NovikovThorneDisk(inner_radius=6.0, outer_radius=18.0)
configurations = VacuumOTEConfigurations(spacetime=spacetime,
                                   camera = camera,
                                   radiative_model = model,
                                   unit_mass_in_solar_masses=1.0)
initial_data = initialize(configurations)
cb, cbp = callback_setup(configurations; rhorizon_bound=2e-1) #... or, define your own cb and cbp
run = integrate(initial_data, configurations, cb, cbp; method=VCABM(), reltol=1e-5, abstol=1e-5)
output_data = run.output_data

Equations integration stats:
Wall clock time: 0.344285111 seconds
Memory allocated: 78.439 MiB


8×2500 Matrix{Float64}:
 -205.347       -205.396       -205.449       …  -207.116       -207.063
   12.7801        12.5039        12.2301           12.3734        12.6369
    1.5708         1.5708         1.5708            1.5708         1.5708
   -0.790152      -0.769363      -0.747727          2.38835        2.36846
   -1.17974       -1.18459       -1.18967          -1.18694       -1.1822
   -0.207056      -0.211034      -0.215492    …    -0.107213      -0.104482
    0.0827336      0.0846515      0.0866524         0.0871341      0.0851671
   -0.00520887    -0.00522038    -0.00522468        0.00531839     0.00531333

In [70]:
#N=600 chunks 2
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=2)
nothing

  0.149714 seconds (230.40 k allocations: 26.587 MiB)


  0.005891 seconds (1.84 k allocations: 5.663 MiB)


In [76]:
#N=600 chunks 1
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=1)
nothing

  0.142022 seconds (230.40 k allocations: 26.587 MiB, 11.36% gc time)
  0.000313 seconds (26 allocations: 5.496 MiB)


In [77]:
#N=600 chunks 3
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=3)
nothing

  0.140230 seconds (230.40 k allocations: 26.587 MiB)
  0.003948 seconds (66 allocations: 5.501 MiB)


In [ ]:
#N=200 chunks 2
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=2)
nothing

  0.007765 seconds (14.12 k allocations: 1.903 MiB)


  0.000476 seconds (46 allocations: 630.484 KiB)


In [ ]:
#N=200 chunks 1
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=1)
nothing

  0.005976 seconds (14.12 k allocations: 1.903 MiB)
  0.000077 seconds (36 allocations: 629.047 KiB)


In [ ]:
#N=200 chunks 3
@time observed_bolometric_intensities(initial_data, output_data, configurations, camera)
@time observed_bolometric_intensities_multithreading(initial_data, output_data, configurations, camera; chunks_per_thread=3)
nothing

  0.008128 seconds (14.12 k allocations: 1.903 MiB)
  0.000282 seconds (71 allocations: 633.812 KiB)


In [ ]:
#N=50 chunks 2
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  316.410 μs (905 allocations: 123.83 KiB)


  7.151 μs (50 allocations: 45.25 KiB)


In [16]:
#N=50 chunks 3
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=3)
nothing

  316.190 μs (905 allocations: 123.83 KiB)


  12.428 μs (70 allocations: 47.91 KiB)


In [15]:
#N=50 chunks 1
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=1)
nothing

  316.049 μs (905 allocations: 123.83 KiB)


  5.442 μs (25 allocations: 41.92 KiB)


In [ ]:
#N=10 chunks 2
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  12.362 μs (41 allocations: 5.55 KiB)


  4.822 μs (48 allocations: 7.78 KiB)


In [ ]:
#N=3
@btime observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera)
@btime observed_bolometric_intensities_multithreading($initial_data, $output_data, $configurations, $camera; chunks_per_thread=2)
nothing

  975.571 ns (9 allocations: 1.05 KiB)


  2.845 μs (48 allocations: 6.28 KiB)
